## Setup

In [ ]:
!git clone https://github.com/CTLab-ITMO/self-expanding-nets -b freeze-layers

Cloning into 'self-expanding-nets'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (541/541), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 541 (delta 300), reused 519 (delta 279), pack-reused 0 (from 0)
Receiving objects: 100% (541/541), 1.96 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (300/300), done.


In [ ]:
!pip install -U -e ./self-expanding-nets/ --force-reinstall

Obtaining file:///content/self-expanding-nets
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.6 MB/s eta 0:00:00
  Using cached setuptools-75.8.2-py3-none-any.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from senmodel.model.utils import convert_dense_to_sparse_network, get_model_last_layer
from senmodel.metrics.edge_finder import EdgeFinder
from senmodel.model.utils import freeze_all_but_last, freeze_only_last
from senmodel.metrics.nonlinearity_metrics import *

from tqdm import tqdm


SEED = 8642

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Data

In [ ]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse',
           'ship', 'truck')

train_dataset = datasets.CIFAR10(root='./data', train=True,
                                  download=True, transform=transform)
val_dataset = datasets.CIFAR10(root='./data', train=False,
                                  download=True, transform=transform)

# train_size = int(0.8 * len(train_dataset))
# val_size = len(train_dataset) - train_size
# # train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size // 2, val_size // 2, len(train_dataset) - (train_size // 2 + val_size // 2)])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

len(train_loader)

## Model

In [ ]:
def freeze_model(model, num_trainable_layers: int = 1):
    for i in range(len(list(model.children())) - num_trainable_layers):
        for param in list(model.children())[i].parameters():
            param.requires_grad = False


def print_layer_status(model):
    for name, param in model.named_parameters():
        print(f"Layer: {name}, frozen: {not param.requires_grad}")

In [ ]:
class ExpandingHead(nn.Module):
    def __init__(self, input_size: int = 64, hidden_size: int = 50, output_size: int = 10):
        super().__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size, output_size)
        # self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        # x = self.relu(self.fc2(x))
        # x = self.dropout(x)
        # x = self.fc3(x)
        return x

In [ ]:
class ResnetExp(nn.Module):
    def __init__(self, freeze_base: bool = False, device=torch.device('cpu')):
        super().__init__()
        self.base_model = torch.hub.load("chenyaofo/pytorch-cifar-models",
                                         "cifar10_resnet20", pretrained=True)
        self.base_model = torch.nn.Sequential(
            *(list(self.base_model.children())[:-1])
        ).to(device)
        self.expanding_head = convert_dense_to_sparse_network(
            ExpandingHead(input_size=64,
                          # hidden_size=50,
                          output_size=10).to(device),
            device=device
        )
        if freeze_base:
            freeze_model(self.base_model)

    def forward(self, x):
        x = self.base_model(x)
        x = x.view(x.size(0), -1)
        x = self.expanding_head(x)
        return x

In [ ]:
device

In [ ]:
model = ResnetExp(freeze_base=True, device='cuda')
input_tensor = torch.randn(1, 3, 32, 32).to('cuda')  # ТУТ ПРОБЛЕМА С ГПУ
output = model(input_tensor)
output

## Train loop

In [ ]:
def edge_replacement_func_new_layer(model, optim, val_loader, metric, choose_threshold, aggregation_mode='mean', len_choose=None):
    layer = get_model_last_layer(model)
    ef = EdgeFinder(metric, val_loader, device, aggregation_mode)

    vals = ef.calculate_edge_metric_for_dataloader(model, len_choose, False)
    print("Edge metrics:", vals, max(vals, default=0), sum(vals))

    chosen_edges = ef.choose_edges_threshold(model, choose_threshold, len_choose)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))

    layer.replace_many(*chosen_edges)

    if len(chosen_edges[0]) > 0:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
    else:
        print("Empty metric")

    return {'max': max(vals, default=0), 'sum': sum(vals), 'len': len(vals), 'len_choose': layer.count_replaces[-1]}

In [ ]:
def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None,
                           window_size=3, threshold=0.1, lr=5e-4, choose_threshold=0.3, aggregation_mode='mean', replace_all_epochs=3):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    val_losses = []

    len_choose = get_model_last_layer(model).count_replaces

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        optimizer.zero_grad()


        for i, (inputs, targets) in enumerate(tqdm(train_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()

            if len(len_choose) > replace_all_epochs and i > window_size:
                freeze_all_but_last(model)

            optimizer.step()
            optimizer.zero_grad()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_targets, all_preds)
        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

        new_l = dict()

        val_losses.append(val_loss)
        # print(edge_replacement_func)
        # print(len(val_losses), val_losses, window_size)
        if edge_replacement_func and len(val_losses) > window_size:
            print("checking if edge replacement is needed")
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            print(f"{recent_changes}")
            avg_change = sum(recent_changes) / window_size
            print(f"avg_change = {avg_change}, threshold = {threshold}, win_size = {window_size}")
            if avg_change < threshold:
                print(f"{len_choose=}")
                len_ch = len_choose[-1] if len(len_choose) > replace_all_epochs else None
                new_l = edge_replacement_func(model, optimizer, val_loader, metric, choose_threshold, aggregation_mode, len_ch)
                # Замораживаем все слои кроме последнего
                val_losses = []
                len_choose = get_model_last_layer(model).count_replaces

        wandb.log({'val_loss': val_loss, 'val_accuracy': val_accuracy, 'train_loss': train_loss} | new_l)

## Training

In [ ]:
criterion = nn.CrossEntropyLoss()
metrics = [
    MagnitudeL2Metric(criterion),
    SNIPMetric(criterion),
    # GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

hyperparams = {"num_epochs": 50,
               "metric": metrics[0],
               "aggregation_mode": "mean",
               "choose_threshold": 0.08,
               "window_size": 100,
               "threshold": 0.08,
               "lr": 5e-4,
               "replace_all_epochs": 100
               }

In [ ]:
sparse_model = ResnetExp(freeze_base=False, device=device)
sparse_model = sparse_model.to(device)

print_layer_status(sparse_model)

In [ ]:
import wandb

wandb.init(
    project="self-expanding-nets",
    name="resnet full expanding, no freeze, threshold = choose_threshold = 0.08"
)

In [ ]:
train_sparse_recursive(sparse_model, train_loader, val_loader,
                       edge_replacement_func=edge_replacement_func_new_layer,
                       **hyperparams)

In [ ]:
wandb.finish()